<a href="https://colab.research.google.com/github/hildazaqya/project-capstone/blob/main/ML/Foodection_Pra_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#this file from kaggle API personnal account
from google.colab import files

# Upload kaggle.json
files.upload()

Saving kaggle.json to kaggle (3).json


{'kaggle (3).json': b'{"username":"hildaz","key":"f0531584456defa83c8102f0f937ac8d"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
##link dataset in Kaggle.com
!kaggle datasets download -d alinesellwia/food-freshness

food-freshness.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
#new directory for stores temporary data
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [5]:
#unzip the data
!unzip -q food-freshness.zip -d /content/data

replace /content/data/Foodection Dataset/test/jeruk_busuk/jeruk_busuk (1).png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


Resize Gambar Untuk Satu Pixel Yang Sama

In [6]:
#Train Data, Augmentation
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Model, regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D, Dropout

#Transfer Learning
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications import EfficientNetB0

#optimizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop

#interach with OS -> make directory
import os

#evaluation
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

#Visualization
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

#for save model
from tensorflow.keras.models import load_model
from google.colab import files


In [7]:
base_dir= '/content/data/Foodection Dataset'

print("Contents of base directory:")
print(os.listdir(base_dir))

print("\nContents of train directory:")
print(os.listdir(f'{base_dir}/train'))

print("\nContents of validation directory:")
print(os.listdir(f'{base_dir}/test'))

Contents of base directory:
['test', 'train']

Contents of train directory:
['wortel_busuk', 'tomat_segar', 'tomat_busuk', 'jeruk_segar', 'tomat_segar_sedang', 'wortel_segar_sedang', 'wortel_segar', 'jeruk_busuk', 'jeruk_segar_sedang']

Contents of validation directory:
['wortel_busuk', 'tomat_segar', 'tomat_busuk', 'jeruk_segar', 'wortel_segar_sedang', 'wortel_segar', 'jeruk_busuk', 'jeruk_segar_sedang', 'tomat _segar_sedang']


In [8]:
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

# Specify class of the directory train
train_jeruksegar_dir = os.path.join(train_dir, 'jeruk_segar')
train_wortelsegar_dir = os.path.join(train_dir, 'wortel_segar')
train_wortelsegarsedang_dir = os.path.join(train_dir, 'wortel_segar')
train_tomatbusuk_dir = os.path.join(train_dir, 'tomat_busuk')
train_jeruksegarsedang_dir = os.path.join(train_dir, 'jeruk_segar_sedang')
train_jerukbusuk_dir = os.path.join(train_dir, 'jeruk_busuk')
train_wortelbusuk_dir = os.path.join(train_dir, 'wortel_busuk')
train_tomatsegar_dir = os.path.join(train_dir, 'tomat_segar')
train_tomatsegarsedang_dir = os.path.join(train_dir, 'tomat_segar_sedang')

# Specify class of the directory test
test_jeruksegar_dir = os.path.join(test_dir, 'jeruk_segar')
test_wortelsegar_dir = os.path.join(test_dir, 'wortel_segar')
test_wortelsegarsedang_dir = os.path.join(test_dir, 'wortel_segar_sedang')
test_tomatbusuk_dir = os.path.join(test_dir, 'tomat_busuk')
test_jeruksegarsedang_dir = os.path.join(test_dir, 'jeruk_segar_sedang')
test_jerukbusuk_dir = os.path.join(test_dir, 'jeruk_busuk')
test_wortelbusuk_dir = os.path.join(test_dir, 'wortel_busuk')
test_tomatsegar_dir = os.path.join(test_dir, 'tomat_segar')
test_tomatsegarsedang_dir = os.path.join(test_dir, 'tomat _segar_sedang')

In [9]:
train_jeruksegar_fnames = os.listdir(train_jeruksegar_dir)
train_wortelsegar_fnames = os.listdir(train_wortelsegar_dir)
train_wortelsegarsedang_fnames = os.listdir(train_wortelsegarsedang_dir)
train_tomatbusuk_fnames = os.listdir(train_tomatbusuk_dir)
train_jeruksegarsedang_fnames = os.listdir(train_jeruksegarsedang_dir)
train_jerukbusuk_fnames = os.listdir(train_jerukbusuk_dir)
train_wortelbusuk_fnames = os.listdir(train_wortelbusuk_dir)
train_tomatsegar_fnames = os.listdir(train_tomatsegar_dir)
train_tomatsegarsedang_fnames = os.listdir(train_tomatsegarsedang_dir)

In [10]:
print('total training jeruk segar images :', len(os.listdir(train_jeruksegar_dir)))
print('total training wortel segar images :', len(os.listdir( train_wortelsegar_dir)))
print('total training wortel segar sedang images :', len(os.listdir(train_wortelsegarsedang_dir)))
print('total training tomat busuk images :', len(os.listdir(train_tomatbusuk_dir)))
print('total training jeruk segar sedang images :', len(os.listdir(train_jeruksegarsedang_dir)))
print('total training jeruk busuk images :', len(os.listdir(train_jerukbusuk_dir)))
print('total training wortel busuk images :', len(os.listdir(train_wortelbusuk_dir)))
print('total training tomat segar images :', len(os.listdir(train_tomatsegar_dir)))
print('total training tomat segar sedang images :', len(os.listdir(train_tomatsegarsedang_dir)))

print('total testing jeruk segar images :', len(os.listdir(test_jeruksegar_dir)))
print('total testing wortel segar images :', len(os.listdir(test_wortelsegar_dir)))
print('total testing wortel segar sedang images :', len(os.listdir(test_wortelsegarsedang_dir)))
print('total testing tomat busuk images :', len(os.listdir(test_tomatbusuk_dir)))
print('total testing jeruk segar sedang images :', len(os.listdir(test_jeruksegarsedang_dir)))
print('total testing jeruk busuk images :', len(os.listdir(test_jerukbusuk_dir)))
print('total testing wortel busuk images :', len(os.listdir(test_wortelbusuk_dir)))
print('total testing tomat segar images :', len(os.listdir(test_tomatsegar_dir)))
print('total testing tomat segar sedang images :', len(os.listdir(test_tomatsegarsedang_dir)))

total training jeruk segar images : 3460
total training wortel segar images : 4241
total training wortel segar sedang images : 4241
total training tomat busuk images : 1115
total training jeruk segar sedang images : 2401
total training jeruk busuk images : 1658
total training wortel busuk images : 1360
total training tomat segar images : 4745
total training tomat segar sedang images : 3109
total testing jeruk segar images : 1482
total testing wortel segar images : 1818
total testing wortel segar sedang images : 966
total testing tomat busuk images : 478
total testing jeruk segar sedang images : 1029
total testing jeruk busuk images : 703
total testing wortel busuk images : 583
total testing tomat segar images : 2033
total testing tomat segar sedang images : 1332


In [36]:
from PIL import Image

# Path to the directory containing the original images
input_directory = os.path.join(base_dir, 'train')
# Path to the directory where you want to save resized images
output_directory = os.path.join(base_dir, 'train_resize')

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# List all files in the input directory
files = os.listdir(input_directory)

# Desired dimensions for resizing
resized_dimensions = (128, 128)

# Loop through each file and resize
for file in files:
    # Construct the full file path
    input_path = os.path.join(input_directory, file)

    # Check if the entity is a file (not a directory)
    if os.path.isfile(input_path):
        # Open the image using PIL
        img = Image.open(input_path)

        # Resize the image to the desired dimensions
        resized_img = img.resize(resized_dimensions)

        # Construct the output file path
        output_path = os.path.join(output_directory, file)

        # Save the resized image
        resized_img.save(output_path)

print("Image resizing complete.")

Image resizing complete.


In [39]:
# Print the contents of the test_resize directory
print("Contents of test_resize directory:")
for root, dirs, files in os.walk(test_dir_resize):
    print(f"{root} contains {len(files)} files")

Contents of test_resize directory:
/content/data/Foodection Dataset/test_resize contains 0 files


In [38]:
from PIL import Image
import os

# Path to the directory containing the original images
input_directory = os.path.join(base_dir, 'test')
# Path to the directory where you want to save resized images
output_directory = os.path.join(base_dir, 'test_resize')

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)

# Desired dimensions for resizing
resized_dimensions = (128, 128)

# Loop through each file and resize
for file in os.listdir(input_directory):
    # Construct the full file path
    input_path = os.path.join(input_directory, file)

    # Check if the entity is a file (not a directory)
    if os.path.isfile(input_path):
        # Open the image using PIL
        img = Image.open(input_path)

        # Resize the image to the desired dimensions
        resized_img = img.resize(resized_dimensions)

        # Construct the output file path
        output_path = os.path.join(output_directory, file)

        # Save the resized image
        resized_img.save(output_path)

print("Image resizing complete.")

Image resizing complete.


In [37]:
# Print the contents of the test_resize directory
print("Contents of test_resize directory:")
for root, dirs, files in os.walk(train_dir_resize):
    print(f"{root} contains {len(files)} files")

Contents of test_resize directory:
/content/data/Foodection Dataset/train_resize contains 0 files


In [33]:
train_dir_resize = os.path.join(base_dir, 'train_resize')
test_dir_resize = os.path.join(base_dir, 'test_resize')

In [41]:
img_height = 128
img_width = 128
img_channels = 3
num_epochs = 20

train_datagen = ImageDataGenerator( rotation_range=20,
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.1,
  zoom_range=0.1,
  horizontal_flip=True,
  vertical_flip=True,
  rescale=1./255,
  validation_split=0.2)

test_datagen  = ImageDataGenerator( rescale = 1.0/255.)

train_generator= train_datagen.flow_from_directory(train_dir,
                                                    batch_size=20,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                    target_size=(img_height, img_width))
test_generator =  test_datagen.flow_from_directory(test_dir,
                                                    batch_size=20,
                                                    color_mode='rgb',
                                                    class_mode='categorical',
                                                    target_size=(img_height, img_width))

Found 24341 images belonging to 9 classes.
Found 10422 images belonging to 9 classes.


In [42]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, img_channels))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Choose a specific layer for feature extraction
x = layers.Flatten()(base_model.output)
x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(9, activation='softmax')(x)  # Adjust the number of output classes

model = Model(base_model.input, x)

#early stop
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

9406464/9406464 [==============================] - 0s 0us/step


In [43]:
history = model.fit(train_generator, epochs=num_epochs, validation_data=test_generator, callbacks=[early_stopping])

Epoch 1/20
 779/1218 [==================>...........] - ETA: 1:12 - loss: 1.8620 - accuracy: 0.7438

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


1218/1218 [==============================] - 287s 232ms/step - loss: 1.5929 - accuracy: 0.7682 - val_loss: 1.6599 - val_accuracy: 0.5932
Epoch 2/20
1218/1218 [==============================] - 252s 207ms/step - loss: 0.9910 - accuracy: 0.8200 - val_loss: 1.7981 - val_accuracy: 0.5686
Epoch 3/20
1218/1218 [==============================] - 254s 209ms/step - loss: 0.9538 - accuracy: 0.8197 - val_loss: 1.7649 - val_accuracy: 0.5856


In [14]:
model1 = tf.keras.models.Sequential()

model1.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model1.add(layers.MaxPooling2D((2, 2)))

model1.add(layers.Conv2D(64, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2, 2)))

model1.add(layers.Conv2D(128, (3, 3), activation='relu'))
model1.add(layers.MaxPooling2D((2, 2)))

model1.add(layers.Flatten())

model1.add(layers.Dense(256, activation='relu'))
model1.add(layers.Dense(9,activation='softmax'))

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [15]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Compile the model
model1.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
history = model1.fit(train_generator, epochs=num_epochs, validation_data=test_generator, callbacks=[early_stopping])

Epoch 1/20
1218/1218 [==============================] - 477s 389ms/step - loss: 1.0686 - accuracy: 0.6167 - val_loss: 1.8854 - val_accuracy: 0.4616
Epoch 2/20
1218/1218 [==============================] - 448s 368ms/step - loss: 0.6640 - accuracy: 0.7684 - val_loss: 2.1513 - val_accuracy: 0.4770
Epoch 3/20
1218/1218 [==============================] - 444s 365ms/step - loss: 0.5419 - accuracy: 0.8116 - val_loss: 2.1108 - val_accuracy: 0.5197
